#Degrees of Separation
##Developing a graph representation of name relationships + BFS

On 23andme.com, you can get info on other DNA relatives of yours who have also signed up on 23andMe. 23andMe provides a csv filed, with all relatives above some threshold of genetic similarity. I have 838 "DNA relatives" with whom I share at least 0.14% DNA (at least according to 23andme).

In [1]:
import pandas as pd
my_rel = pd.read_csv('relative_finder_ME.csv')

In [2]:
my_fam_names ='horstman, nussbeck, lamont, boyd, harris'
#omitting Smith cause it is way too common
my_rel.ix[len(my_rel),'Family Surnames']= my_fam_names
rows = my_rel['Family Surnames'].dropna()
rows.tail(2)

831    Andersen, Teer, Zell, Ostbye, Lamb, Roe, Surbe...
839             horstman, nussbeck, lamont, boyd, harris
Name: Family Surnames, dtype: object

# Data Cleanup
People can voluntarily provide genealogical info, such as Family Surnames.  Unfortunately, this data is quite messy, so I first clean it up a bit.

In [3]:
import re

def _stripstr(s):
    s2 = re.sub(r',|\(|\)|&|father=|/|-|[\s,]du\s|[\s,]de\s|[\s,]di\s|^\w\s|\s\w\s|paternal|maternal|\sjr[\s\.,]|jr$', r' ',s.lower())
    return s2
def _stripword(s):
    """This function removes non-letter characters from a word
    
    for example _strip('Hi there!') == 'Hi there'
    """
    return re.sub(r'[\s\W_]+','', s)

## NEED TO CHECK, make sure this isn't screwing up Swedish letters
# test my the text cleaning
s= '(Fall), Paternal & s  Miller-Rhodes, Rhode Jr, Seal Jr. , di Stefano Jr'
for word in _stripstr(s).split():
    print _stripword(word)

fall
miller
rhodes
rhode
seal
stefano


#Setting up the NameGraph
(Implemented as a dict of lists)
Each family surname represents a possible edge between two "rows."
We use a set of edges to avoid redundant edges.
For now I'll ignore complications such as spelling differences or the fact that two people with same surname may be unrelated. 

**I'll define two people as linked if they share a surname from their 'family surnames' list.**  This will define a graph.  In this project, I want to see who I am "connected" to in this graph.  The data is small enough that I can represent the graph with a 


In [4]:
from collections import defaultdict, Counter

namelist_dict = defaultdict(list)
namecount_dict = Counter()
edges = set({})

ncomp = 0
for (idx,v) in rows.iteritems():
    for word in _stripstr(v).split():
        ncomp +=1
        key = _stripword(word)
        namecount_dict[key]+=1
        if namecount_dict[key] > 1:
            for otheridx in namelist_dict[key]:
                if otheridx != idx:
                    edges.add( (otheridx,idx) )
                    edges.add( (idx,otheridx) )
        namelist_dict[key].append(idx)
    
# should be of order V+E
#print ncomp
#print len(edges) + len(rows)

In [5]:
# How many of my relatives give Smith as a family surname ?
print namecount_dict['smith']
print namecount_dict['yoder']

17
4


In [6]:
#given the set of edges above, it is simple to create the NameGraph
NameGraph = defaultdict(list)
for (a,b) in edges:
    NameGraph[a].append(b)

In [7]:
#I am 839.  Which people am I directly linked (by family surnames)?
print NameGraph[839]
print rows.ix[839]

[554, 433, 168, 766, 9, 558, 1, 730, 542, 0]
horstman, nussbeck, lamont, boyd, harris


#Time for Breadth First Search!
I adapted this using code from MIT opencourseware :)

In [8]:
from collections import deque

class BFSresult:
    def __init__(self):
        self.level = {}
        self.parent = {}

def bfs(g,s):
    
    r = BFSresult()
    r.level = {s: 0}
    r.parent = {s: None}
    
    q = deque()
    q.append(s)
    
    while q:
        u = q.popleft()
        for n in g[u]:
            if n not in r.parent:
                r.parent[n] = u
                r.level[n] = r.level[u] + 1
                q.append(n)
    return r
                
    

In [9]:
my_rel['degree_of_sep'] = pd.Series(bfs(NameGraph,839).level)

my_rel['DNAsim'] = my_rel['DNA Shared'].replace('%','',regex=True).astype('float')/100
my_rel[my_rel['degree_of_sep']==1].DNAsim

0      0.0229
1      0.0150
9      0.0039
168    0.0025
433    0.0018
542    0.0017
554    0.0016
558    0.0016
730    0.0015
766    0.0015
Name: DNAsim, dtype: float64

In [10]:
print my_rel[my_rel['degree_of_sep'] == 1].DNAsim.mean()
print my_rel[my_rel['degree_of_sep'] == 2].DNAsim.mean()
print my_rel[my_rel['degree_of_sep'] == 3].DNAsim.mean()

0.0054
0.00201870503597
0.00189714285714


#Next steps:
1) Handling different spellings of the same name

2) Giving the edges weights.   Common names should be worth less. Weights should take into consideration genetic relationship

3) Dijkstra's Algorithm to find shortest paths (closest relatives)
